In [1]:
import mysql.connector
import pandas as pd
from kafka import KafkaProducer
from datetime import datetime
import time
import random
import numpy as np

In [3]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "root",
    database = "client_rw"
)

In [4]:
cursor = mydb.cursor()
query = "select * from transaction_summary"
cursor.execute(query)
result = cursor.fetchall()
columns = [col[0] for col in cursor.description]
df = pd.DataFrame(result, columns=columns)

In [5]:
df = df[["Account_No","TRANSACTION_DETAILS","Cheq_No","WITHDRAWL_AMOUNT","DEPOSIT_AMT","BALANCE_AMT"]]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116201 entries, 0 to 116200
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Account_No           116201 non-null  object
 1   TRANSACTION_DETAILS  116201 non-null  object
 2   Cheq_No              116201 non-null  object
 3   WITHDRAWL_AMOUNT     116201 non-null  object
 4   DEPOSIT_AMT          116201 non-null  object
 5   BALANCE_AMT          116201 non-null  object
dtypes: object(6)
memory usage: 5.3+ MB


In [7]:
'''
name of the Kafka topic that the consumer will subscribe to and read messages from
'''
KAFKA_TOPIC_NAME_CONS = "Topic4"
'''
indicates that the Kafka broker is running on the local machine and is accessible through port 9092
'''
KAFKA_BOOTSTRAP_SERVERS_CONS = 'localhost:9092'

In [8]:
if __name__ == "__main__":
    print("Kafka Producer Application Started ... ")

    kafka_producer_obj = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS_CONS,
                                       value_serializer=lambda x: x.encode('utf-8'))
    
    df =df.head(50)
    df['id'] = np.arange(len(df)) 
    df_list = df.to_dict(orient="records")
    message_list = []
    message = None
    for message in df_list:
        
        message_fields_value_list = []
               
        message_fields_value_list.append(message["id"])
        message_fields_value_list.append(message["Account_No"])
        # message_fields_value_list.append(message["DATE"])
        message_fields_value_list.append(message["TRANSACTION_DETAILS"])
        message_fields_value_list.append(message["Cheq_No"])
        # message_fields_value_list.append(message["VALUE_DATE"])
        message_fields_value_list.append(message["WITHDRAWL_AMOUNT"])
        message_fields_value_list.append(message["DEPOSIT_AMT"])
        message_fields_value_list.append(message["BALANCE_AMT"])

        message = ','.join(str(v) for v in message_fields_value_list)
        print("Message Type: ", type(message))
        print("Message: ", message)
        kafka_producer_obj.send(KAFKA_TOPIC_NAME_CONS, message)
        time.sleep(1)


    print("Kafka Producer Application Completed. ")

Kafka Producer Application Started ... 
Message Type:  <class 'str'>
Message:  0,409000611074',TRF FROM  Indiaforensic SERVICES,,0.00,1000000.00,1000000.00
Message Type:  <class 'str'>
Message:  1,409000611074',TRF FROM  Indiaforensic SERVICES,,0.00,1000000.00,2000000.00
Message Type:  <class 'str'>
Message:  2,409000611074',FDRL/INTERNAL FUND TRANSFE,,0.00,500000.00,2500000.00
Message Type:  <class 'str'>
Message:  3,409000611074',TRF FRM  Indiaforensic SERVICES,,0.00,3000000.00,5500000.00
Message Type:  <class 'str'>
Message:  4,409000611074',FDRL/INTERNAL FUND TRANSFE,,0.00,500000.00,6000000.00
Message Type:  <class 'str'>
Message:  5,409000611074',FDRL/INTERNAL FUND TRANSFE,,0.00,500000.00,6500000.00
Message Type:  <class 'str'>
Message:  6,409000611074',FDRL/INTERNAL FUND TRANSFE,,0.00,500000.00,7000000.00
Message Type:  <class 'str'>
Message:  7,409000611074',FDRL/INTERNAL FUND TRANSFE,,0.00,500000.00,7500000.00
Message Type:  <class 'str'>
Message:  8,409000611074',FDRL/INTERNAL